In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# # Rename Columns
# df = pd.read_csv('Dataset/dataset_cleaned.csv', low_memory=False)

# column_mapping = {
#     'Make': 'Maker',
#     'number_of_cylinders': 'Number_of_Cylinders',
#     'engine_type': 'Engine_Type',
#     'engine_hp': 'Engine_Horse_Power',
#     'engine_hp_rpm': 'Engine_Horse_Power_RPM',
#     'transmission': 'Transmission',
#     'mixed_fuel_consumption_per_100_km_l': 'Mixed_Fuel_Consumption_per_100_km_l',
#     'fuel_tank_capacity_l': 'Fuel_Tank_Capacity',
#     'acceleration_0_100_km/h_s': 'Acceleration_0_to_100_Km',
#     'max_speed_km_per_h': 'Max_Speed_Km_per_Hour',
#     'fuel_grade': 'Fuel_Grade',
#     'year': 'Year',
#     'Tipe_Mobil': 'Type_of_Car'
# }

# # Rename columns using the mapping
# df.rename(columns=column_mapping, inplace=True)

# df.to_csv('Dataset/dataset_cleaned1.csv', index=False)

In [3]:
column = ['Number_of_Cylinders',
          'Engine_Type',
          'Engine_Horse_Power',
          'Engine_Horse_Power_RPM',
          'Transmission',
          'Acceleration_0_to_100_Km',
          'Fuel_Grade',
          'Mixed_Fuel_Consumption_per_100_km_l']

target = 'Mixed_Fuel_Consumption_per_100_km_l'

df = pd.read_csv('Dataset/dataset_cleaned1.csv', usecols=column, low_memory=False)
df.head()

,Number_of_Cylinders,Engine_Type,Engine_Horse_Power,Engine_Horse_Power_RPM,Transmission,Mixed_Fuel_Consumption_per_100_km_l,Acceleration_0_to_100_Km,Fuel_Grade
0,8,0,354.0,6500,1,13.4,5.6,3
1,8,0,326.0,5800,1,12.2,6.1,3
2,6,0,240.0,5750,0,13.2,8.7,2
3,6,0,290.0,6200,0,10.4,7.6,3
4,6,0,255.0,6800,1,10.0,5.0,3


**TEST MODEL**

This method is based on tensorflow's mpg regression: https://www.tensorflow.org/tutorials/keras/regression

In [4]:
train_data = df.sample(frac=0.8, random_state=0)
test_data = df.drop(train_data.index)
print(f"Train data: {len(train_data)}")
print(f"Test data: {len(test_data)}")

Train data: 14876
Test data: 3719


In [5]:
train_features = train_data.copy()
test_features = test_data.copy()

train_labels = train_features.pop(target)
test_labels = test_features.pop(target)

In [6]:
train_data.describe().transpose()[['mean', 'std']]

,mean,std
Number_of_Cylinders,4.657368,1.353355
Engine_Type,0.334902,0.471972
Engine_Horse_Power,173.874899,93.959119
Engine_Horse_Power_RPM,5221.490992,1006.727885
Transmission,0.588734,0.492080
Mixed_Fuel_Consumption_per_100_km_l,7.769505,2.608764
Acceleration_0_to_100_Km,10.128542,3.069313
Fuel_Grade,1.905351,1.385076


In [7]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

[[4.6573682e+00 3.3490193e-01 1.7387497e+02 5.2214941e+03 5.8873391e-01
  1.0128551e+01 1.9053503e+00]]


In [8]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units=1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_absolute_error',
              metrics=['mae'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 7)                15        
 n)                                                              
                                                                 
 dense (Dense)               (None, 1)                 8         
                                                                 
Total params: 23
Trainable params: 8
Non-trainable params: 15
_________________________________________________________________


In [9]:
history = model.fit(train_features, train_labels, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
372/372 [==============================] - 3s 4ms/step - loss: 7.5663 - mae: 7.5663 - val_loss: 7.4716 - val_mae: 7.4716
Epoch 2/50
372/372 [==============================] - 1s 3ms/step - loss: 7.1940 - mae: 7.1940 - val_loss: 7.1001 - val_mae: 7.1001
Epoch 3/50
372/372 [==============================] - 1s 3ms/step - loss: 6.8219 - mae: 6.8219 - val_loss: 6.7283 - val_mae: 6.7283
Epoch 4/50
372/372 [==============================] - 1s 3ms/step - loss: 6.4500 - mae: 6.4500 - val_loss: 6.3568 - val_mae: 6.3568
Epoch 5/50
372/372 [==============================] - 1s 3ms/step - loss: 6.0778 - mae: 6.0778 - val_loss: 5.9850 - val_mae: 5.9850
Epoch 6/50
372/372 [==============================] - 1s 3ms/step - loss: 5.7057 - mae: 5.7057 - val_loss: 5.6133 - val_mae: 5.6133
Epoch 7/50
372/372 [==============================] - 1s 3ms/step - loss: 5.3336 - mae: 5.3336 - val_loss: 5.2415 - val_mae: 5.2415
Epoch 8/50
372/372 [==============================] - 1s 3ms/step - loss: 4.

lol, loss sama mae nya sama nilainya tiap epoch. sus

In [10]:
model.save("model_test1.h5")

In [11]:
# Assuming you have a Normalization layer named 'normalizer'
saved_mean = normalizer.mean.numpy()
saved_variance = normalizer.variance.numpy()

# Save the mean and variance to a file
np.save('saved_mean.npy', saved_mean)
np.save('saved_variance.npy', saved_variance)

In [19]:
normalizer1 = tf.keras.layers.Normalization(axis=-1, input_shape=(train_features.shape[1],))
normalizer1.adapt(np.array(train_features))

In [ ]:
model1 = tf